ETL PROCESAMIENTO SISTEMATICO CERTIFICADOS ANALITICOS MLP

PROCESAMIENTO INFO CERTIFICADOS

In [ ]:
"""@author: Francisco Donaire"""

#IMPORTAMOS LAS LIBRERIAS NECESARIAS
import pandas as pd
import openpyxl
import os
import datetime


directory_path = 'G:\\.shortcut-targets-by-id\\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\\09. Contrato CS115 EIA 2024\\03.WIP\\03.Hidroquímica\\2. Actualizacion BDD\\Certificados Revisados\\Otros certificados\\Version4'


data_certificados = pd.DataFrame()

excel_files = [file for file in os.listdir(directory_path) if file.endswith(('.xlsx'))]

# Procesar cada archivo Excel encontrados
for file in excel_files:
    file_path = os.path.join(directory_path, file)  # Construye la ruta completa al archivo
    xls = pd.ExcelFile(file_path)
    wb = openpyxl.load_workbook(file_path)

    #DETERMINAMOS LA EXTENSION HORIZONTAL DE MANERA AUTOMATICA
    for hoja_archivo in xls.sheet_names:
        hoja = wb[hoja_archivo]
        rango = hoja['C133:AZ133']

        ultima_celda_con_datos = None
        for celda in rango[0]:
            if celda.value is not None:
                ultima_celda_con_datos = celda

        if ultima_celda_con_datos:
            columna = ''.join(filter(str.isalpha, ultima_celda_con_datos.coordinate))
            print(f'La última columna con datos es {columna}, en la hoja: {hoja_archivo}, del archivo: {file}')
    
    
            data_identificacion = pd.read_excel(xls, sheet_name= hoja_archivo, skiprows=132, nrows=8, usecols=f"C:{columna}")


            data_identificacion_transposed = data_identificacion.transpose()


            data_identificacion_transposed.columns = data_identificacion_transposed.iloc[0]
            data_identificacion_transposed = data_identificacion_transposed.drop(data_identificacion_transposed.index[0])
            
            # ASIGNACION DE NOMBRES A LAS COLUMNAS
            data_identificacion_transposed.columns = ['Fecha', 'Id_Certificado', 'Comentarios', 'Año_QAQC', 'Fecha_QAQC', 'Mes_QAQC', 'Muestras_Con_Datos', 'Muestras_Totales']
            data_identificacion_transposed['Index_Column'] = data_identificacion_transposed.index

            # RESET DEL INDICE PARA OBTENER UNO NUEVO E INCREMENTAL
            data_identificacion_transposed.reset_index(drop=True, inplace=True)
            
            # AGREGA COLUMNA CON LA RUTA DEL ARCHIVO
            data_identificacion_transposed['Ruta_Archivo'] = file_path
            data_identificacion_transposed.columns = ['Fecha', 'Id_Certificado', 'Comentarios', 'Año_QAQC', 'Fecha_QAQC', 'Mes_QAQC', 'Muestras_Con_Datos', 'Muestras_Totales', 'Estacion', 'Ruta_Archivo']
            
            # ORDEN DE LA COLUMNAS 
            data_identificacion_transposed = data_identificacion_transposed[['Estacion','Fecha', 'Id_Certificado', 'Ruta_Archivo', 'Comentarios', 'Año_QAQC', 'Fecha_QAQC', 'Mes_QAQC', 'Muestras_Con_Datos', 'Muestras_Totales']]
            
            # SE LIMPIA COLUMNA "ESTACION"
            data_identificacion_transposed['Estacion'] = data_identificacion_transposed['Estacion'].str.replace(r'\..*$', '', regex=True)
            
            # SE UNE LA DATA NUEVA AL DATAFRAME PRINCIPAL
            data_certificados = pd.concat([data_certificados, data_identificacion_transposed], ignore_index=True)
            
            
        else:
            print(f'No hay datos en el rango C133:AZ133, para la hoja: {hoja_archivo}, para el archivo: {file}')




IMPRIMIR DATA PARA REVISAR CANTIDAD DE DATOS 

In [ ]:
data_certificados

PROCESAR DATA PARAMETROS

In [ ]:
import pandas as pd
import openpyxl
import os

directory_path = 'G:\\.shortcut-targets-by-id\\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\\09. Contrato CS115 EIA 2024\\03.WIP\\03.Hidroquímica\\2. Actualizacion BDD\\Certificados Revisados\\Otros certificados\\Version4'

data_parametros = pd.DataFrame()

# ENCONTRAR ARCHIVOS EXCEL EN EL DIRECTORIO
excel_files = [file for file in os.listdir(directory_path) if file.endswith(('.xlsx'))]

# LEER CADA ARCHIVO EXCEL ENCONTRADO
for file in excel_files:
    file_path = os.path.join(directory_path, file)  # CONSTRUYE LA RUTA COMPLETA AL ARCHIVO
    xls = pd.ExcelFile(file_path)
    wb = openpyxl.load_workbook(file_path)
    # EVALUA COLUMNAS CON DATOS EN EL RANGO
    for hoja_archivo in xls.sheet_names:
        hoja = wb[hoja_archivo]
        rango = hoja['C133:AZ133']
        # DETERMINA LA ULTIMA COLUMNA CON DATOS
        ultima_celda_con_datos = None
        for celda in rango[0]:
            if celda.value is not None:
                ultima_celda_con_datos = celda
        # SI HAY DATOS EN EL RANGO OBTIENE LA COORDENADA DE LA ULTIMA COLUMNA
        if ultima_celda_con_datos:
            columna = ''.join(filter(str.isalpha, ultima_celda_con_datos.coordinate))
            print(f'La última columna con datos es >>> {columna} <<< , en la hoja: >>> {hoja_archivo} <<<')
    
            # SE LEE EL RANGO DE CELDAS CON LOS DATOS DE LOS PARAMETROS
            data_df = pd.read_excel(xls, sheet_name= hoja_archivo, skiprows=141, nrows=111, usecols=f"B:{columna}", header=None) #
            # data_df.columns = data_df.reset_index()
            data_df.drop(data_df.columns[1], axis=1, inplace=True) #ELIMINA COLUMNA VACIA
            
            # data_df.reset_index(drop=False, inplace=False)
            #PIVOTEO DEL DATAFRAME
            data_pivoted = data_df.T

            # PARAMETROS COMO COLUMNAS, PRIMERA FILA COMO ENCABEZADO
            new_header = data_pivoted.iloc[0]  
            data_pivoted = data_pivoted[1:]  
            data_pivoted.columns = new_header  

            #RESETEA EL INDICE
            data_pivoted.reset_index(drop=True, inplace=True)

            # SE UNE LA NUEVA DATA 
            data_parametros = pd.concat([data_parametros, data_pivoted], ignore_index=True)
            # for column in data_parametros.columns:
            #     data_parametros[column] = pd.to_numeric(data_parametros[column], errors='coerce')
            
        else:
            print(f'No hay datos en el rango C133:AZ133, para la hoja: {hoja_archivo}')
            
         
data_parametros

REVISAR DATA PARAMETROS

In [ ]:
data_parametros

SE JUNTAN AMBOS DATA_FRAME PARA LOGRAR FORMATO COLUMNAR INDEXADO|

In [ ]:
df_merged = pd.concat([data_certificados, data_parametros], axis=1)

SE LIMPIA COLUMNA ESTACION

In [ ]:
# df_merged['Estacion'] = df_merged['Estacion'].str.replace(r'\..*$', '', regex=True)

In [ ]:
df_merged

CREAMOS CONECCION  A LA TABLA EN LA BD

In [ ]:
import psycopg2
from sqlalchemy import create_engine

# Credenciales db
host = 'localhost'
database = 'certificados_dbo'
username = 'postgres'  
password = '1234' 

# Crea la coneccion
conn = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password
)

engine = create_engine('postgresql://'+username+':'+password+'@'+host+'/'+database)
table_name = 'certificados_procesados'

df_data_bd = pd.read_sql_table('certificados_procesados', con=engine)#Trae la tabla de la bda un dataframe

df_data_bd

SE UNE LA DATA PROVENIENTE DE LA BBDD A LA DATA NUEVA QUE SE ACTUALIZARÁ

In [ ]:



df_update = pd.concat([df_data_bd, df_merged], ignore_index=True, axis=0) # se le añade la nueva info
df_update

CARGAMOS A LA BBDD LA NUEVA DATA COMPILADA

In [ ]:


# df_update = df_merged #DESCOMENTAR PARA SOLO INSERTAR UNA TABLA NUEVA

df_update.to_sql(table_name, engine, index=False, if_exists='replace',)

BAJAMOS LA DATA DE LA BBDD A UN DATA-FRAME PARA REVISAR CANTIDADES Y CONSISTENCIA

In [ ]:
table_name = 'certificados_procesados'

df_data_bd_rev = pd.read_sql_table('certificados_procesados', con=engine)  #Trae la tabla de la bda un dataframe


df_data_bd_rev